In [1]:
from pathlib import Path
from tqdm import tqdm
import xarray as xr
import yaml
from PIL import Image

from convml_tt.data.dataset import TRIPLET_TILE_IDENTIFIER_FORMAT

### Make set of tile images from previously sampled locations

In [2]:
# Save directory on Sense group workspace
save_dir = "/gws/nopw/j04/sensecdt/users/flojo/data/"

# Working folder
folder = "case_studies/whole_granules"
data_filepath = Path(save_dir+folder)

In [9]:
tile_size = 64

In [10]:
tiles_dir = "/home/users/flojo/Data/Tiles"
tiles_folder = f'case_studies_whole_granules_{tile_size}px'
tiles_filepath = Path(tiles_dir) / tiles_folder
tiles_meta_filepath = tiles_filepath / "meta"

tiles_images_filepath = tiles_filepath / "images"
tiles_images_filepath.mkdir(exist_ok=True)

In [11]:
with open(tiles_meta_filepath / 'tile_locations.yaml', 'r') as f:
    tile_locations = yaml.safe_load(f)

In [12]:
tile_locations

{'MOD021KM.A2020013.1405': [{'loc': {'latitude': 61.71612548828125,
    'longitude': -58.588443756103516,
    'scene_id': 'MOD021KM.A2020013.1405',
    'x': 32,
    'y': 528},
   'region': 1,
   'tile_id': 0},
  {'loc': {'latitude': 61.6060791015625,
    'longitude': -58.60045623779297,
    'scene_id': 'MOD021KM.A2020013.1405',
    'x': 32,
    'y': 544},
   'region': 1,
   'tile_id': 1},
  {'loc': {'latitude': 61.49601745605469,
    'longitude': -58.61268997192383,
    'scene_id': 'MOD021KM.A2020013.1405',
    'x': 32,
    'y': 560},
   'region': 1,
   'tile_id': 2},
  {'loc': {'latitude': 61.3077392578125,
    'longitude': -58.61486053466797,
    'scene_id': 'MOD021KM.A2020013.1405',
    'x': 32,
    'y': 576},
   'region': 1,
   'tile_id': 3},
  {'loc': {'latitude': 61.19770812988281,
    'longitude': -58.62717819213867,
    'scene_id': 'MOD021KM.A2020013.1405',
    'x': 32,
    'y': 592},
   'region': 1,
   'tile_id': 4},
  {'loc': {'latitude': 61.009761810302734,
    'longitude': 

#### Sliding window tiles

In [13]:
scene_ids = list(tile_locations.keys())
scene_ids

['MOD021KM.A2020013.1405',
 'MOD021KM.A2020040.1525',
 'MYD021KM.A2020013.1555',
 'MYD021KM.A2020040.1535']

In [14]:
for scene_id in tqdm(scene_ids):
    # make individual folders
    image_folder = tiles_images_filepath / scene_id
    image_folder.mkdir(exist_ok=True)
    
    scene_tiles = tile_locations[scene_id]
    granule_folder  = data_filepath / "granule_data"
    granule_data = xr.open_dataset(granule_folder / f"{scene_id}.nc")
    for tile in scene_tiles:
        tile_id = tile['tile_id']
        x_c, y_c = tile['loc']['x'], tile['loc']['y']

        tile_rgb = granule_data["true_color"].isel(x=slice(x_c-tile_size//2, x_c+tile_size//2), 
                                                            y=slice(y_c-tile_size//2, y_c+tile_size//2))
        img = Image.fromarray(tile_rgb.values)
        resized_img = img.resize((256,256), Image.Resampling.NEAREST)

        tile_image_path = image_folder / f"{scene_id}_{tile_id:05d}.png"
        resized_img.save(tile_image_path)
        


100%|██████████| 4/4 [02:45<00:00, 41.36s/it]


#### Monthly tiles

In [6]:
for month in tqdm(range(1,13)):
    for satellite in ['aqua', 'terra']:
        tiles = tile_locations[month][satellite]
        granule_folder = data_filepath / str(month) / satellite / "granule_data"
        for scene_id in tiles.keys():
            granule_data = xr.open_dataset(granule_folder / f"{scene_id}.nc")
            for tile in tiles[scene_id]:
                x_c, y_c = tile['loc']['x_c'], tile['loc']['y_c']
                triplet_id = tile['triplet_id']
                tile_type = tile['tile_type']

                tile_rgb = granule_data["true_color"].isel(x=slice(x_c-tile_size//2, x_c+tile_size//2), 
                                                            y=slice(y_c-tile_size//2, y_c+tile_size//2))
                img = Image.fromarray(tile_rgb.values)
                resized_img = img.resize((256,256), Image.Resampling.NEAREST)
                tile_identifier = TRIPLET_TILE_IDENTIFIER_FORMAT.format(triplet_id=triplet_id, 
                                                                                tile_type=tile_type)
                fn_image = f"{tile_identifier}.png"
                resized_img.save(tiles_images_filepath / fn_image)

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [22:26<00:00, 112.22s/it]
